In [30]:
import pandas as pd
from dateutil.parser import parse
pd.options.display.max_rows = 20
import os, glob
import pandas_profiling

## 文件遍历方法
### 方法一：os.walk(dirpath)
<br>

```  
for dirpath, subdirs, files in os.walk(fd_path):
    for f in files:
        if f.endswith('csv'):
            pass
```

### 方法二： glob.glob(pattern)
<br>

```
all_files = glob.glob(os.path.join(fd_path, '*.csv'))
for f in all_files:
    pass
```

In [31]:
fd_path = r"C:\Users\5106001995\Desktop\2019_pypj\_34.pd_np_plt\_src01_so85_log"
if os.path.isdir(fd_path) and fd_path !='':
    all_files = sorted(glob.glob(os.path.join(fd_path, '*.csv')))

## 读取方法
### .read_csv()

|参数|结果|
|-----|-----|
|.read_csv(f)|报错,每行的列数不一样导致 |
|.read_csv(f, header=None, usecols=range(22), names=['h'+str(i) for i in range(22)])|指定需要的行数，并命名列|


In [32]:
col_names = ['h'+str(i) for i in range(22)]
# li = []
# for f in all_files:
#     df = pd.read_csv(f,
#                     header=None,
#                     usecols=range(22),
#                     names=col_names)
#     li.append(df)
    
# big_df = pd.concat(li, axis=0, ignore_index=True)
big_df = pd.concat((pd.read_csv(f,header=None,usecols=range(22),names=col_names) for f in all_files), ignore_index=True)

In [33]:
big_df.head()

,h0,h1,h2,h3,h4,h5,h6,h7,h8,h9,...,h12,h13,h14,h15,h16,h17,h18,h19,h20,h21
0,20190322055715,RC_VCOM,1100,RC,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,20190322064542,RC_VCOM,1100,RC,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,20190322071156,RC_VCOM,1100,RC,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,20190322074321,RC_VCOM,1100,RC,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,20190322074912,RC_VCOM,1100,RC,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [34]:
usecols = ['h0', 'h3', 'h6', 'h13', 'h17', 'h21']
new_col_names = {'h0':'datetime', 'h3':'pmod', 'h6':'scc', 'h13':'x', 'h17':'y', 'h21':'Y'}
output_df = pd.DataFrame(big_df, columns=usecols).rename(columns=new_col_names, inplace=False) #method1
# output_df = big_df.loc[:,usecols] #method2
# output_df = big_df[usecols] #method3
# output_df = big_df.iloc[:,[0,3,6,13,17,21]] #method4
# output_df.columns = output_df.columns.map(lambda x: new_col_names[x])
# pd.value_counts(output_df['y'].isna())
output_df = output_df[(output_df['pmod'] != 'RC') & (output_df['y'] > 0) & (output_df['Y'] > 10)]
output_df.reset_index(drop=True).head()

,datetime,pmod,scc,x,y,Y
0,20190322153017,A5000142A,55.0,0.2830,0.2901,922.5
1,20190322153628,A5000142A,54.0,0.2839,0.2931,950.0
2,20190322160550,A5000142A,58.0,0.2851,0.2966,969.4
3,20190323061234,A5000142A,103.0,0.2859,0.2968,999.5
4,20190325025455,A5000142A,241.0,0.2848,0.2990,889.4


# 如何将df.astype()
将int转换成str?
- <font color=blue>√<font color=black> 思路一: df.apply(str)
- <font color=red>x<font color=black> 思路二: df.apply(lambda x: str(x))
- <font color=red>x<font color=black> 思路三: df['datetime'].to_datetime()
- <font color=red>x<font color=black> 思路四: df.astype({'datetime':'object'})
- <font color=blue>√<font color=black> 思路五: dateutil.parser.parse()
    
<br>注意：因为字符串是immutable，需要把.apply(str)后的结果再次assign给原df['datetime']，否则不起作用

In [35]:
# def convert_datetime(i):
#     s = str(i)
#     y,m,d = s[0:4],s[4:6],s[6:8]
#     hh,mm,ss = s[8:10],s[10:12],s[12:14]
#     return '{0}-{1}-{2} {3}:{4}:{5}'.format(y,m,d,hh,mm,ss)
# output_df['datetime'] = output_df['datetime'].apply(convert_datetime) #method1
# output_df['datetime'] =  pd.to_datetime(output_df['datetime'])

output_df['datetime'] = pd.to_datetime(output_df['datetime'].apply(lambda x: parse(str(x)))) #method2
output_df['scc'] = output_df['scc'].apply(lambda x: "{:0>7}".format(int(x)))
# output_df.iloc[0,0]
output_df.reset_index(drop=True).head()

,datetime,pmod,scc,x,y,Y
0,2019-03-22 15:30:17,A5000142A,0000055,0.2830,0.2901,922.5
1,2019-03-22 15:36:28,A5000142A,0000054,0.2839,0.2931,950.0
2,2019-03-22 16:05:50,A5000142A,0000058,0.2851,0.2966,969.4
3,2019-03-23 06:12:34,A5000142A,0000103,0.2859,0.2968,999.5
4,2019-03-25 02:54:55,A5000142A,0000241,0.2848,0.2990,889.4


In [36]:
output_path1 = os.path.join(os.path.split(fd_path)[0], 'SO85_output.csv')
output_path2 = os.path.join(os.path.split(fd_path)[0], 'SO85_output.html')
output_df.to_csv(output_path1, index=False)
pandas_profiling.ProfileReport(output_df).to_file(output_path2)

# 字符串的占位符placehold
口诀：**填对宽, 分精类**
```
scc = 100
form = '{:0>7}'.format(scc)
form
'0000100'
```